In [4]:
%matplotlib inline
import os
import pandas as pd
from fbprophet import Prophet
import json
from datetime import datetime, timedelta

In [22]:
def get_datetime_from_string(s, keep_utc=True):
    if keep_utc:
        return datetime.strptime(s + "00", '%Y-%m-%d %H:%M:%S%z')
    else:
        return datetime.strptime(s[:-3], '%Y-%m-%d %H:%M:%S')

In [1]:
abbreviations = {'Baden-Württemberg': 'BW', 'Bayern': 'BY', 'Berlin': 'BE', 'Brandenburg': 'BB', 'Bremen': 'HB', 'Hamburg': 'HH', 'Hessen': 'HE', 'Mecklenburg-Vorpommern': 'MV', 'Niedersachsen': 'NI', 'Nordrhein-Westfalen': 'NW', 'Rheinland-Pfalz': 'RP', 'Saarland': 'SL', 'Sachsen': 'SN', 'Sachsen-Anhalt': 'ST', 'Schleswig-Holstein': 'SH', 'Thüringen': 'TH', 'unknown': 'unknown'}

In [8]:
# Export vacation information
for _, state_name in abbreviations.items():
    if state_name == 'unknown':
        continue
    years = []
    names = []
    for year in range(2014, 2019):
        year_content = open(os.path.join('..', 'data', 'external', "vacations_{}_{}.json".format(year, state_name))).read()
        year_dict = json.loads(year_content)
        for date in year_dict["daten"]:
            start = datetime.fromtimestamp(date["beginn"])
            end = datetime.fromtimestamp(date["ende"])
            delta = (end - start).days
            for vacation_date in range(0, delta + 1):
                years.append(start + timedelta(days=vacation_date))
                names.append(date["title"].split(' ')[0])
    df_vacations = pd.DataFrame({
      'holiday': names,
      'ds': pd.to_datetime(years),
    })
    df_vacations.to_csv(os.path.join('..', 'data', 'processed', "vacations_{}.csv".format(state_name)))
df_vacations.head()

,ds,holiday
0,2014-04-14,Osterferien
1,2014-04-15,Osterferien
2,2014-04-16,Osterferien
3,2014-04-17,Osterferien
4,2014-04-18,Osterferien


In [12]:
# Export holiday information
for _, state_name in abbreviations.items():
    if state_name == 'unknown':
        continue
    years = []
    names = []
    for year in range(2014, 2019):
        year_content = open(os.path.join('..', 'data', 'external', "holidays_{}.json".format(year))).read()
        year_dict = json.loads(year_content)
        for day in year_dict[state_name].keys():
            years.append(year_dict[state_name][day]['datum'])
            names.append(day)
    df_holidays = pd.DataFrame({
      'holiday': names,
      'ds': pd.to_datetime(years),
    })
    df_holidays.to_csv(os.path.join('..', 'data', 'processed', "holidays_{}.csv".format(state_name)))
df_holidays.head()

,ds,holiday
0,2014-11-01,Allerheiligen
1,2014-06-09,Pfingstmontag
2,2014-06-19,Fronleichnam
3,2014-04-18,Karfreitag
4,2014-12-25,1. Weihnachtstag
